In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate

2024-01-15 00:41:29.060871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Select GPU number 1
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPUs


2024-01-15 00:42:09.585468: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-15 00:42:09.595966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-15 00:42:10.240797: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-15 00:42:10.240958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-15 00:42:10.241059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [10]:
# 生成虛擬數據作為示例
num_samples = 10000
num_items = 20000
user_ids = np.random.randint(1, 1000, size=num_samples)
item_ids = np.random.randint(1, num_items, size=num_samples)
labels = np.random.choice([0, 1], size=(num_samples, num_items), p=[0.5, 0.5])

In [12]:
# 將數據劃分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(
    np.column_stack((user_ids, item_ids)),
    labels,
    test_size=0.2,
    random_state=42
)

In [23]:
class DeepCrossNetwork(tf.keras.Model):
    def __init__(self, input_dim, embedding_dim, cross_layer_num, hidden_units, num_items):
        super(DeepCrossNetwork, self).__init__()

        self.embedding_layer = Embedding(input_dim=input_dim, output_dim=embedding_dim)
        self.flatten = Flatten()

        self.deep_layers = [Dense(units, activation='relu') for units in hidden_units]
        self.cross_layers = [self.cross_layer for _ in range(cross_layer_num)]

        self.concatenate = Concatenate()
        self.output_layer = Dense(num_items, activation='sigmoid')


    def call(self, inputs):
        user_ids, item_ids = inputs

        embedding_user = self.embedding_layer(user_ids)
        embedding_item = self.embedding_layer(item_ids)

        flat_embedding_user = self.flatten(embedding_user)
        flat_embedding_item = self.flatten(embedding_item)

        deep_output = flat_embedding_user
        for layer in self.deep_layers:
            deep_output = layer(deep_output)

        cross_output = flat_embedding_user
        for layer in self.cross_layers:
            cross_output = layer(cross_output, flat_embedding_item)

        combined_output = self.concatenate([deep_output, cross_output])
        output = self.output_layer(combined_output)

        return output

    def cross_layer(self, x0, x):
        cross = tf.keras.layers.Dot(axes=1)([x, x0])
        return x + cross

In [25]:
# Example usage
num_items = 20000  # Adjust based on your dataset
input_dim = 2  # Adjust based on your dataset
embedding_dim = 16
cross_layer_num = 3
hidden_units = [64, 32]

model = DeepCrossNetwork(input_dim, embedding_dim, cross_layer_num, hidden_units, num_items)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

dummy_input = [tf.constant([[1], [2]]), tf.constant([[3], [4]])]  # Replace with actual data
model(dummy_input)
model.summary()

# Train the model
model.fit([X_train[:, 0], X_train[:, 1]], y_train, epochs=20, batch_size=32, validation_split=0.2)


Model: "deep_cross_network_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     multiple                  32        
                                                                 
 flatten_6 (Flatten)         multiple                  0         
                                                                 
 dense_17 (Dense)            multiple                  1088      
                                                                 
 dense_18 (Dense)            multiple                  2080      
                                                                 
 concatenate_5 (Concatenate)  multiple                 0         
                                                                 
 dense_19 (Dense)            multiple                  980000    
                                                                 
Total params: 983,200
Trainable params: 983,20

: 